### 创建数据

In [39]:
import numpy as np
import pandas as pd
import math
from math import log

In [26]:
# 建立数据集/特征向量
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

In [27]:
datasets, labels = create_data()

### 信息熵

In [28]:
def calc_shannon_ent(dataset):
    n = len(dataset) # 计算长度
    label_counts = {} # 建立一个字典存储
    # 统计每个类别出现的次数
    for feature in dataset:
        label = feature[-1]
        if label not in label_counts:
            label_counts[label] = 0     # 创建该元素并清零
        label_counts[label] += 1
    entropy = 0  # 存储信息熵
    for key in label_counts:
        p = float(label_counts[key]) / n  # 计算类概率，或者说类在所有数据中的比例
        entropy -= p * log(p, 2)
    return entropy

In [64]:
calc_shannon_ent(datasets)

0.9709505944546686

In [65]:
# 在某指定维度上对dataset进行划分，抽取某指定维度上等于value的特征
def splite_dataset(dataset, axis, value):
    splt_dset = []  # 定义一个列表用来存储value
    for f in dataset:
        if f[axis] == value:
            reduce_fv = f[:axis]  # 此维度之前的所有列
            reduce_fv.extend(f[axis+1:])  # 列表末尾插入此维度之后的所有列
            splt_dset.append(reduce_fv)
    return splt_dset

In [66]:
splite_dataset(datasets, 1, '是')

[['青年', '否', '好', '是'],
 ['青年', '是', '一般', '是'],
 ['中年', '是', '好', '是'],
 ['老年', '否', '好', '是'],
 ['老年', '否', '非常好', '是']]

In [67]:
# 计算数据集中的主要类别，这里计算是借的多还是不借的多，其实就是判断yes多还是no多
# 其他应用中可能会出现多个类别标签，因此这是一个通用函数
def majority_cnt(classlist):
    classcount = {}
    for label in classlist:
        if label not in classcount.keys():  # 字典的键中不包含label
            classcount[label] = 0
        classcount[label] += 1
    # classcount.items()是以list形式返回字典的键值：dict_items([('yes', 2), ('no', 4)])
    # key中指定值获取函数，x[1]表示用键值对第二个参数来排序
    # reverse表示降序排序，默认是ascending
    sorted_class_count = sorted(classcount.items(), key=lambda x: x[1], reverse=True)
    return sorted_class_count[0][0]  # 返回出现次数最多的类的标签

In [68]:
classlist = [f[-1] for f in datasets] # 从数据集中抽取最后一列的值
print(classlist)
majority_cnt(classlist)

['否', '否', '是', '是', '否', '否', '否', '是', '是', '是', '是', '是', '是', '是', '否']


'是'

In [69]:
# 通过遍历所有的特征，求取熵最小的划分方式
# 返回划分数据最好的特征，和最大的信息增益
def min_entropy_split_feature(dataset):
    f_n = len(dataset[0]) - 1  # 特征数
    base_entropy = calc_shannon_ent(dataset)  # 信息熵
    best_info_gain = 0.0  # 定义变量存储最佳信息增益
    #print("entropy calc:"+str(dataset))
    best_feature = -1
    for i in range(f_n):
        f_list = [feature[i] for feature in dataset] # 抽取数据集中第i+1个特征的值
        unique_values = set(f_list)  # 去除该列重复的值
        new_entropy = 0.0  # 存储条件熵
        for value in unique_values:
            sub_dataset = splite_dataset(dataset, i, value)  # 在某指定维度上对dataset进行划分，抽取某指定维度上等于value的特征
            pro = len(sub_dataset) / float(len(dataset)) # 计算某指定特征中，每一个value的概率
            new_entropy += pro * calc_shannon_ent(sub_dataset) # 计算条件熵
        info_gain = base_entropy - new_entropy  # 信息增益是信息熵的减小量，也就是：信息熵-条件熵
        if info_gain > best_info_gain:    # 条件判断，若信息增益大于最佳信息增益
            best_info_gain = info_gain
            best_feature = i
    return best_feature, best_info_gain

In [70]:
min_entropy_split_feature(datasets)

(2, 0.4199730940219749)

### 利用ID3算法生成决策树

In [71]:
# 递归构建决策树
def create_tree(dataset, __labels):
    labels = __labels.copy()
    classlist = [f[-1] for f in dataset]
    # 只剩下一个类了，因此返回类名称
    if classlist.count(classlist[0]) == len(classlist): 
        return classlist[0]
    # 数据集中只剩下最后一列了，也就是所有的特征都用完了，没法再向下分类了
    # 这时候如果数据集中有多个类，那就认为是出现最多的那个类
    # 实际中应该是，所给定的特征无法将数据进行完全分类导致的
    if len(dataset[0]) == 1:
        # print("******")
        return majority_cnt(classlist)
    bestfeature = min_entropy_split_feature(dataset)[0] # 返回最好的特征
    # if bestfeature >= len(labels):
    #     print("**************index out of range")
    #     print(dataset)
    #     print(labels)
    #     print(bestfeature)
    bestf_label = labels[bestfeature]
    newtree = {bestf_label: {}} # 创建一个字典存储树，将best future作为根节点
    del labels[bestfeature] # 删除该类别
    f_values = [f[bestfeature] for f in dataset]  # 找出bestfeature全部取值
    unique_f_values = set(f_values) # 去掉重复的值
    for v in unique_f_values:
        sublabels = labels[:]
        _splitedtree = splite_dataset(dataset, bestfeature, v) # 在某指定维度上对dataset进行划分，抽取某指定维度上等于value的特征
        # print("xxxxxxxxxxxxxx seperate:")
        # print("best fv:" + str(bestfeature) + " v:" + str(v))
        # print(_splitedtree)
        # print(labels)
        newtree[bestf_label][v] = create_tree(_splitedtree.copy(), sublabels) # 完成递归，在每个分支再调用函数
    return newtree

In [72]:
tree = create_tree(datasets, labels)
print(tree)

{'有自己的房子': {'是': '是', '否': {'有工作': {'是': '是', '否': '否'}}}}


In [34]:
# 用构建好的决策树来进行分类
def classify(input_tree, feature_labels, test_fv):
    classlabel = "none"

    first_label = list(input_tree.keys())[0]  # 获取字典中的第一个键位
    new_dict = input_tree[first_label]  # 返回key中的value
    feature_index = feature_labels.index(first_label)  # 获取first_label是第几个特征
    for key in new_dict.keys():
        if test_fv[feature_index] == key:  # 注意，key是特征的值
            if type(new_dict[key]).__name__ == "dict":  # 如果key中的value类型为dict
                classlabel = classify(new_dict[key], feature_labels, test_fv)# 进入下一分支
            else:
                classlabel = new_dict[key] # 返回标签
    return classlabel

In [73]:
x = classify(tree, labels, ['青年', '否', '否', '一般'])
print(x)

否
